# 공용 함수
- 수정하지 않는 것을 추천드립니다

In [283]:
from jy import get_all
import pandas as pd
from neuralprophet import NeuralProphet

In [284]:
# 상대경로 지정 없이 원본 csv 임포트
main_df = get_all()
main_df_columns = main_df.columns
date_df = main_df.loc[:,"date"]

In [285]:
# 각 지역구와 관련된 변수들
# common은 지역 구분 없는 전체 변수
gw_vars = main_df.loc[:,main_df_columns[2:9]]
ddm_vars = main_df.loc[:,main_df_columns[9:16]]
sd_vars = main_df.loc[:,main_df_columns[16:23]]
jl_vars = main_df.loc[:,main_df_columns[23:30]]
common_vars = main_df.loc[:,main_df_columns[30:46]]

In [286]:
# 각 지역구의 따릉이 수요량 
gw_demand=main_df.loc[:,main_df_columns[46]]
ddm_demand=main_df.loc[:,main_df_columns[47]]
sd_demand=main_df.loc[:,main_df_columns[48]]
jl_demand=main_df.loc[:,main_df_columns[49]]

In [287]:
# 날짜와 수요량 컬럼을 원하는 대로 정리해주는 함수
def create_y (df_demand, date_name, y_name):
    result = pd.concat([date_df, df_demand], axis=1)
    result = result.rename(columns={"date":date_name, result.columns[1]:y_name})
    return result

In [288]:
# 변수들을 일부 샘플링한 dataframe의 리스트를 반환하는 함수
## amount는 (수요량 이외에) 추가로 샘플링할 변수의 개수 (0, 1, 2 가능. 0은 수요량만 반환)
## df_y는 날짜와 수요량이 포함된 데이터프레임. create_y 함수로 정제한 값을 넣으면 됩니다
## df_list는 추가 변수로 사용할 모든 데이터프레임들의 리스트. ~~~_vars 중 원하는 것을 리스트로 넣으면 됩니다
    ## 예시) 광진구 + 공용 변수의 경우: df_list=[gw_vars, comon_vars]
## 반환하는 값은 튜플 {"data": 데이터프레임, "name":포함된 컬럼명의 리스트} 로 이루어진 리스트입니다
    
def sample_multivariate(amount, df_y, df_list):
    samples = []
    # 데이터프레임을 하나로 통합
    df = pd.concat(df_list, axis=1)
    if amount == 0:
        # y값만 리턴
        samples.append({"data":pd.concat([df_y], axis=1),"name":[]})
    elif amount == 1:
        # sample을 한 column씩 추출
        for index in range(len(df.columns)):
            sample = pd.concat([df_y,df.loc[:,df.columns[index]]],axis=1)
            samples.append({"data":sample, "name":[df.columns[index]]})
    elif amount == 2:
        # sample을 두 column씩 추출
        for first_index in range(len(df.columns)):
            remaining_columns = df.columns[first_index+1:]
            for second_index in range(len(remaining_columns)):
                targets = [df.columns[first_index],remaining_columns[second_index]]
                sample = pd.concat([df_y,df.loc[:,targets]],axis=1)
                samples.append({"data":sample, "name":targets})
    return samples
    

In [289]:
# 데이터를 자르는 함수
# 일단은 2021년 1월 1일 기준으로 자르게 일반화 해놓았습니다

def split_sample(df):
    train = df.loc[:1095]
    test = df.loc[1096:]
    return {"train":train, "test":test}


In [298]:
def create_result(indexes):
    return pd.DataFrame(index=["Column"] + indexes)

공용 함수 End

# neural prophet을 이용한 광진구 test 입니다
- 윤호님 모델을 거의 그대로 사용
- 광진구 관련 변수를 1개씩 샘플링해서 테스트


In [290]:
# create_y로 광진구 수요량 데이터 정제
gw_y = create_y(gw_demand,"ds","y")


In [291]:
# 광진구 관련 데이터(gw_vars)에서 1개씩 sampling하여 test_dfs 제작
test_dfs = sample_multivariate(1,gw_y,[gw_vars])

In [297]:
# test_dfs가 제대로 샘플링 되었는 지 확인
# 총 6개의 변수가 있으므로 6개의 데이터프레임이 포함된 리스트가 생성되었을 겁니다
for test_df in test_dfs:
    print(test_df["data"].head(1))

         ds      y  air_GW
0  1/1/2018  0.592    42.0
         ds      y  real_pop_GW
0  1/1/2018  0.592      421.214
         ds      y  pop_GW
0  1/1/2018  0.592  357785
         ds      y  pop_male_GW
0  1/1/2018  0.592       161143
         ds      y  pop_female_GW
0  1/1/2018  0.592         174428
         ds      y  mvpop_GW
0  1/1/2018  0.592    260642
         ds      y  station_GW
0  1/1/2018  0.592          37


In [301]:
# create_result 함수로 result 생성
result = create_result(["MAE","MAE_val"])
# result가 잘 생성되었는 지 확인

print(result)

""
Column
MAE
MAE_val


In [302]:
# for문을 통해 모델 생성
for index, test_df in enumerate(test_dfs):

    # 샘플링 된 컬럼명
    columns = test_df["name"]

    # test_df로부터 test, train 데이터셋 분리
    splitted = split_sample(test_df["data"])
    test = splitted["test"]
    train = splitted["train"]

    # 모델 생성
    m = NeuralProphet(
        growth='linear', # 추세 유형 설정(linear, discontinuous, off 중 선택 가능)
        yearly_seasonality=True, #년간 계절성 설정
        weekly_seasonality=True, #주간 계절성 설정
        daily_seasonality=True, #일간 계절성 설정
        batch_size=32,#배치 사이즈 설정
        epochs=30,#학습 횟수 설정
        learning_rate=0.1, # 학습률 설정
    )
    # 독립 변인(변수) 추가 및 정규화
    # regressor의 names로는 샘플링된 컬럼명인 columns를 사용했습니다
    m = m.add_lagged_regressor(names=columns, normalize="minmax") 
    m.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    # 학습 수행
    metrics = m.fit(train, freq='h', validation_df=test)
    
    # 기록
    # result의 첫 행은 Column 명입니다!
    result[index] = [columns,metrics.loc[29,"MAE"],metrics.loc[29,"MAE_val"]]





INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.909% of the data.
WARNING - (NP.df_utils._infer_frequency) - Defined frequency h is different than major frequency D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
d:\Tool\Python\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:225: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
d:\Tool\Python\lib\site-packages\pytorch_lightning\trainer\trainer.py:1892: PossibleUserWarning: The number of training batches (35) is smaller than t

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [304]:
# result 확인
print(result)


                0              1         2              3                4  \
Column   [air_GW]  [real_pop_GW]  [pop_GW]  [pop_male_GW]  [pop_female_GW]   
MAE      1.242584       1.328464  1.240033       1.304153         1.215533   
MAE_val  2.994706       3.573744  3.106534       3.275686          2.34856   

                  5             6  
Column   [mvpop_GW]  [station_GW]  
MAE        1.248657      1.240757  
MAE_val    2.968355      2.339918  
